<a href="https://colab.research.google.com/github/ZSCode768/Machine-Learning-Project/blob/main/Surlow_EDA_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook's purpose is to perform exploratory data analysis (EDA) on the "Game Recommendations on Steam" data set from kaggle. (https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam)  The focus will be to understand the structure of the data, generate descriptive statistics,
handle missing values, and create visualizations.

## Key Objectives:
1. Load the dataset and examine its structure.
2. Perform basic cleaning and handle missing values.
3. Generate descriptive statistics.
4. Create visualizations to uncover insights.

In [43]:
# Import necessary libraries.
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount("/content/drive")

# Load the Dataset
file_path = '/content/drive/My Drive/datasets/game_recs/recommendations.csv'
data = pd.read_csv(file_path, delimiter=',', on_bad_lines='skip')

# Display first few rows and dataset info
print("Data head")
print(data.head())
print("\n\n")
print("Data info")
print(data.info())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-43-3467b3121550>:12: DtypeWarning: Columns (0,1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, delimiter=',', on_bad_lines='skip')


Data head
    app_id helpful  funny        date is_recommended  hours   user_id  \
0   975370       0    0.0  2022-12-12           True   36.3   51580.0   
1   304390       4    0.0  2017-02-17          False   11.5    2586.0   
2  1085660       2    0.0  2019-11-17           True  336.5  253880.0   
3   703080       0    0.0  2022-09-23           True   27.4  259432.0   
4   526870       0    0.0  2021-01-10           True    7.9   23869.0   

   review_id  
0        0.0  
1        1.0  
2        2.0  
3        3.0  
4        4.0  



Data info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3513134 entries, 0 to 3513133
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   app_id          object 
 1   helpful         object 
 2   funny           float64
 3   date            object 
 4   is_recommended  object 
 5   hours           float64
 6   user_id         float64
 7   review_id       float64
dtypes: float64(4), object(4)
memory usage: 21

In [44]:
# Check for missing values
print("Missing values")
print(data.isnull().sum())

# Basic stats for # columns
print("\n\n")
print("Basic stats for numerical columns")
print(data.describe())

Missing values
app_id             0
helpful            2
funny             10
date              12
is_recommended    16
hours             24
user_id           30
review_id         39
dtype: int64



Basic stats for numerical columns
              funny         hours       user_id     review_id
count  3.513124e+06  3.513110e+06  3.513104e+06  3.513095e+06
mean   1.503962e+00  2.024429e+02  7.280668e+06  6.005821e+05
std    4.812376e+02  1.074025e+03  4.175900e+06  3.382727e+05
min    0.000000e+00  0.000000e+00  2.000000e+00  0.000000e+00
25%    0.000000e+00  3.100000e+01  3.851423e+06  3.078160e+05
50%    0.000000e+00  1.003000e+02  7.222203e+06  6.005830e+05
75%    0.000000e+00  2.848000e+02  1.110101e+07  8.933485e+05
max    6.362570e+05  1.136152e+06  1.153374e+09  8.767845e+06


In [45]:
# Standardize is_recommend values to eliminate data corruption
data['is_recommended'] = data['is_recommended'].str.lower()
data['is_recommended'] = data['is_recommended'].map({'true': True, 'false': False})
data = data.dropna(subset=['is_recommended'])

# Display the is_recommended values after standardization
print(data['is_recommended'].value_counts())
print("\n\n")
# Display Percentages
percen = data['is_recommended'].value_counts(normalize=True)*100
percen = percen.map(lambda x: f"%{x:.6f}")
print(percen)

is_recommended
True     664823
False    121595
Name: count, dtype: int64



is_recommended
True     %84.538121
False    %15.461879
Name: proportion, dtype: object
